### Preprocessing Inputs of Pre-trained Models

In [14]:
# OpenCV read in BGR format, the loaded images are similar to a numpy array

import cv2
import numpy as np
import urllib

POSE_IMAGE = cv2.imread('images/img1.jpg')

# TODO: shape -> [1x3x256x456] - format [BxCxHxW]
def pose_estimation(input_image):
    image = np.copy(input_image)
    dim = (456, 256) 
    print("original shape {}".format(image.shape))
    image = cv2.resize(image, dim)
    image = image.transpose(2, 0, 1)
    print('preprocessing shape image {}'.format(image.shape))
    image = image.reshape(1, 3, 256, 456)
    return image
    
def text_detection(input_image):
    '''
    using cv2.resize()
    '''
    preprocessed_image = np.copy(input_image)

    preprocessed_image = cv2.resize(preprocessed_image, (1280, 768))
    preprocessed_image = preprocessed_image.transpose((2,0,1))
    preprocessed_image = preprocessed_image.reshape(1, 3, 768, 1280)
    return preprocessed_image
    
def main():
    #choose model
    img = pose_estimation(POSE_IMAGE)
    
if __name__ == "__main__":
    main()


original shape (513, 693, 3)
preprocessing shape image (3, 256, 456)


In [ ]:
# there are 2 types of datasets: https://software.intel.com/en-us/openvino-toolkit/documentation/pretrained-models
# Public Model Set = must be run through the Model Optimizer, have a original models to train an fine-tuning
# Free Model Set = converted to Intermediate Rep formate, don not have original model, to get with Model Downloader

################### Using Model Downloader tool ######################
# Human Pose Estimation: All precision levels

#cd /opt/intel/openvino/deployment_tools/tools/model_downloader
#sudo ./downloader.py —name vehicle-attributes-recognition-barrier-0039 —precisions INT8 -o /home/workspace
# sudo ./downloader.py --name text-detection-0004 --precisions FP16 -o /home/workspace